# Init

In [1]:
import sys ; sys.path.append('/hosthome/Workspace/Python/Datasets/Twinews')

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
from systemtools.hayj import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.location import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *

In [4]:
from twinews.utils import *

In [5]:
logger = Logger(tmpDir('logs') + "/twinews-indexing.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [6]:
TEST = False
initialDatasetVersion = 1 if TEST else 1
datasetVersion = "1.0"

In [7]:
dataRootPath = dataDir() + "/Twinews/" + "twinews" + str(initialDatasetVersion)

In [8]:
newsFiles = sortedGlob(dataRootPath + "/news/*.bz2")
bp(newsFiles)
bp(list(NDJson(random.choice(newsFiles)))[0].keys(), 5)
bp(list(NDJson(random.choice(newsFiles)))[0]['scrap'].keys(), 5)
if TEST:
    newsFiles = newsFiles[:10]

[
  /NoSave/Data/Twinews/twinews1/news/9news.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/10news.com.bz2,
  ...,
  /NoSave/Data/Twinews/twinews1/news/zdnet.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/zerohedge.com.bz2
]
dict_keys(['domain', 'lastUrlDomain', 'scrap', 'url', 'title', 'redirected', 'lastUrl'])
dict_keys(['meta_data', 'publish_date', 'meta_description', 'authors', 'title', 'meta_lang', 'text', 'tags', 'meta_favicon'])


In [9]:
usersFiles = sortedGlob(dataRootPath + "/users/*.bz2")
bp(usersFiles)
bp(list(NDJson(random.choice(usersFiles)))[0].keys(), 5)

[ /NoSave/Data/Twinews/twinews1/users/part0.bz2, /NoSave/Data/Twinews/twinews1/users/part1.bz2 ]
dict_keys(['tweet_count', 'location', 'bio', 'tweets', 'list_count', 'joindate_title', 'joindate_text', 'moment_count', 'notBotScore', 'verified', 'username', 'user_website', 'follower_count', 'following_count', 'favorite_count', 'user_id', 'datasetRelevanceScore', 'url', 'name', 'media_count', 'tweets_en_ratio', 'consecutive_old_tweets', 'avatar', 'news', 'user_twitter_website', 'date_limit'])


In [10]:
(hjuser, hjpass, hjhost) = getMongoAuth(user='hayj')

In [11]:
newsCollection = getNewsCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)
usersCollection = getUsersCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)

twinews news (version 1.0) initialised.
twinews users (version 1.0) initialised.


In [12]:
newsCollection.resetCollection(security=False)
usersCollection.resetCollection(security=False)

twinews news (version 1.0) initialised.


In [13]:
assert len(newsCollection) == 0
assert len(usersCollection) == 0

In [14]:
tt.tic("Init done")

--> tic: 4.35s | message: Init done


4.35

# News

In [15]:
def preprocessNews(row, logger=None, verbose=True):
    try:
        if isinstance(row, str):
            text = row
        else:
            if dictContains(row, "scrap"):
                row = row['scrap']
            if dictContains(row, "text"):
                text = row['text']
            else:
                raise Exception("No text found in " + b(row))
        text = newsPreclean(text)
        isGood = isGoodArticle(text)
        rawText = text
        (text, tokens) = newsParser.parseNews(rawText, logger=logger, verbose=verbose)
        return (rawText, text, tokens, isGood)
    except Exception as e:
        logException(e, logger, verbose=verbose)

In [16]:
def newsGenFunct(containers, logger=None, verbose=True):
    if not isinstance(containers, list):
        containers = [containers]
    for container in containers:
        for row in NDJson(container, logger=logger, verbose=verbose):
            
            current = dictSelect(row, {'domain', 'lastUrlDomain', 'url', 'title', 'redirected', 'lastUrl'})
            scrap = row['scrap']
            current["title"] = preprocess(current["title"], doRemoveUrls=True, unescapeHtml=True,
                                  removeHtml=True, doQuoteNormalization=True,
                                  doReduceBlank=True, keepNewLines=False, logger=logger)
            (rawText, text, sentences, isGood) = preprocessNews(scrap, logger=logger)
            current["rawText"] = rawText
            current["text"] = text
            current["sentences"] = sentences
            # current["isGoodArticle"] = isGood
            if isGood and text is not None and rawText is not None and sentences is not None and len(sentences) > 0:
                yield current
            else:
                yield None

In [17]:
newsFilesChunks = chunks(newsFiles, int(len(shuffle(newsFiles)) / (cpuCount() * 8)))
bp(newsFilesChunks)

[
  [
    /NoSave/Data/Twinews/twinews1/news/9news.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/10news.com.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/972mag.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/1843magazine.com.bz2
  ],
  [
    /NoSave/Data/Twinews/twinews1/news/abajournal.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/abc.net.au.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/abc15.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/abcactionnews.com.bz2
  ],
  ...,
  [
    /NoSave/Data/Twinews/twinews1/news/yallpolitics.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/yanpritzker.com.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/yorkdispatch.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/yorkshireeveningpost.co.uk.bz2
  ],
  [
    /NoSave/Data/Twinews/twinews1/news/zdnet.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/zerohedge.com.bz2
  ]
]


In [18]:
# We add all news:
mli = MLIterator(newsFilesChunks, newsGenFunct, logger=logger, parallelProcesses=cpuCount(), printRatio=0.001)
notGoodCount = 0
totalCount = 0
duplicatesCount = 0
hashes = set()
for row in mli:
    if row is None:
        notGoodCount += 1
    else:
        h = objectToHash(row["text"])
        if h in hashes:
            duplicatesCount += 1
        hashes.add(h)
        newsCollection.insert(row)
    totalCount += 1
log("We removed " + str(int(notGoodCount / totalCount * 100)) + "% of news.", logger)
log("Count of duplicates: " + str(duplicatesCount), logger)
log("% of duplicates: " + str(duplicatesCount / totalCount * 100), logger)

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
198 containers to process.
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / f

...

 90% [==================  ] (41.055s left)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


 90% [==================  ] (38.752s left)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


 91% [==================  ] (36.438s left)
 91% [==================  ] (34.152s left)
 92% [==================  ] (31.963s left)
 92% [==================  ] (30.002s left)
 93% [==================  ] (27.713s left)
 93% [==================  ] (25.607s left)
 94% [==================  ] (23.455s left)
 94% [==================  ] (21.345s left)
 95% [=================== ] (19.189s left)
 95% [=================== ] (17.196s left)
 96% [=================== ] (15.73s left)
 96% [=================== ] (13.455s left)
 97% [=================== ] (11.267s left)
 97% [=================== ] (8.985s left)
 98% [=================== ] (6.886s left)
 98% [=================== ] (4.63s left)
 99% [=================== ] (2.976s left)
100% [====================] (total duration: 22m 30.46s, mean duration: 6.82s)
We removed 8% of news.
Count of duplicates: 1750
% of duplicates: 7.117582462276813


In [19]:
# We construct a strucutre: hash of the text --> url
newsToKeepByHash = dict()
for _id, row in newsCollection.items():
    h = objectToHash(row["text"])
    newsToKeepByHash[h] = row['url']
newsToKeep = set(newsToKeepByHash.values())
bp(newsToKeepByHash)
assert len(newsToKeep) == len(newsToKeepByHash)
# We construct a structure: the duplicate --> the reference news to take into account
duplicates = dict()
for _id, row in newsCollection.items():
    h = objectToHash(row["text"])
    if newsToKeepByHash[h] != row['url']:
        duplicates[row['url']] = newsToKeepByHash[h]
bp(duplicates)
# We remove duplicates in news:
for dup in duplicates:
    newsCollection.delete({"url": dup})

{
  0006db2155479807dc59d944f54658c3: https://goo.gl/dLfKa5,
  000ea98707fbca374fc16e95def75b95: http://www.bbc.co.uk/news/technology-41928847,
  00115d6f3a322ea9da6cbcd867fe99b1: http://bit.ly/2lSWvxl,
  00162b5b60bcafcb823ba2e780563760: https://www.armytimes.com/off-duty/gearscout/shot-show/2018/01/22/the-rattler-this-ultra-compact-rif,
  00178dc81d34dd31a93004313cbbd333: https://www.rawstory.com/2018/02/minnesota-candidate-gets-anti-muslim-death-threats-militia-movement,
  ...,
  ffedc81ff74d09aa4cbbd88e7efc9c69: https://www.thespec.com/news-story/8100970-court-hears-dead-girl-had-drugs-in-her-system/#.WnElzoryq,
  fff4c675fb5bd94506dfd830430983f4: https://www.nytimes.com/2017/11/17/upshot/income-inequality-united-states.html,
  fffa3f193b0546ec03a7fed794f51cee: https://www.theguardian.com/commentisfree/2017/dec/01/republican-tax-bill-immoral-violence?CMP=Share,
  fffcf8893da78fecee03b51912eb9316: https://www.vice.com/en_us/article/7x9x9z/a-bartender-and-her-favorite-regular-went-so

In [20]:
tt.tic("News indexed")

--> tic: 22m 48.91s | message: News indexed


1368.91

# Users

In [21]:
def usersGenFunct(containers, maxUsersPerContainer=None, logger=None, verbose=True):
    if not isinstance(containers, list):
        containers = [containers]
    for container in containers:
        usersCount = 0
        for row in NDJson(container, logger=logger, verbose=verbose):
            try:
                assert dictContains(row, "tweets")
                # We handle news:
                currentNews = dict()
                for tweet in row["tweets"]:
                    for n in tweet['news']:
                        currentNews[n] = tweet['timestamp']
                newCurrentNews = dict()
                for n, ts in currentNews.items():
                    if n in duplicates:
                        n = duplicates[n]
                    if n in newsToKeep:
                        newCurrentNews[n] = ts
                newCurrentNews = sortBy(newCurrentNews, index=1)
                row["news"] = [e[0] for e in newCurrentNews]
                row["timestamps"] = [e[1] for e in newCurrentNews]
                # We parse tweets:
                for tweet in row["tweets"]:
                    if dictContains(tweet, "text"):
                        tweet["text"] = preprocess\
                        (
                            tweet["text"], logger=logger,
                            doQuoteNormalization=True,
                            doReduceBlank=True,
                            keepNewLines=True,
                            stripAccents=True,
                            doRemoveUrls=True,
                            doLower=False,
                            doBadlyEncoded=True,
                            doReduceCharSequences=True,
                            charSequencesMaxLength=3,
                            replaceUnknownChars=True,
                            unknownReplacer=" ",
                            doSpecialMap=True,
                            doNormalizeEmojis=True,
                            doTokenizingHelp=True,
                        )
                yield row
                usersCount += 1
                if maxUsersPerContainer is not None and usersCount >= maxUsersPerContainer:
                    break
            except Exception as e:
                logException(e, logger)

In [22]:
mli = MLIterator\
(
    usersFiles, usersGenFunct,
    genKwargs={"maxUsersPerContainer": 30 if TEST else None},
    logger=logger, parallelProcesses=cpuCount(),
    printRatio=0.001,
)
for row in mli:
    usersCollection.insert(row)
tt.tic("Users indexed")

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
2 containers to process.
 50% [==========          ] (5m 49.829s left)
100% [====================] (total duration: 6m 11.68s, mean duration: 3m 5.84s)
--> tic: 6m 11.73s | message: Users indexed


371.73

# Adding users references in news

In [24]:
# We iterate all users to make the newsUsersMapping:
newsUsersMapping = dict()
ids = usersCollection.distinct("user_id")
for userId in pb(ids, logger=logger, message="Finding all users by news"):
    data = usersCollection.findOne({"user_id": userId})
    for news in data["news"]:
        if news not in newsUsersMapping:
            newsUsersMapping[news] = []
        newsUsersMapping[news].append(userId)
tt.tic("Users by news collected")

Finding all users by news   0% [                    ]
Finding all users by news  10% [==                  ] (8.729s left)
Finding all users by news  20% [====                ] (7.68s left)
Finding all users by news  30% [======              ] (6.743s left)
Finding all users by news  40% [========            ] (5.775s left)
Finding all users by news  50% [==========          ] (4.75s left)
Finding all users by news  60% [============        ] (3.86s left)
Finding all users by news  70% [==============      ] (2.892s left)
Finding all users by news  80% [================    ] (1.914s left)
Finding all users by news  90% [==================  ] (0.955s left)
Finding all users by news 100% [====================] (total duration: 9.6s, mean duration: 0.019s)
--> tic: 15h 37m 23.699s | message: Users by news collected


56243.7

In [30]:
bp(newsUsersMapping, logger)
countOfGreaterThan1 = 0
total = 0
for url, users in newsUsersMapping.items():
    if len(users) > 1:
        log(url + " --> " + b(users), logger)
        countOfGreaterThan1 += 1
    total += len(users)

{
  http://2020.washingtonexaminer.com/2017/11/14/welcome-trump-county-usa/: [ 252877167 ],
  http://a.msn.com/00/en-us/AAuRcJD?ocid=st: [ 265004300 ],
  http://a.msn.com/00/en-us/AAuyKsY?ocid=st: [ 17000424 ],
  http://a.msn.com/00/en-us/BBFZDuB?ocid=st: [ 145246574 ],
  http://a.msn.com/00/en-us/BBFsnPh?ocid=st: [ 17000424 ],
  ...,
  https://www.zerohedge.com/news/2018-02-14/senior-white-house-aides-say-kelly-pressured-them-lie-him: [ 41368680 ],
  https://xianblog.wordpress.com/2017/03/04/le-bayesianisme-aujourdhui-book-review/: [ 12754522 ],
  https://yaledailynews.com/blog/2017/12/01/law-clinic-helps-block-trump-order/: [ 69194007 ],
  https://yaledailynews.com/blog/2017/12/07/yale-researchers-root-out-latent-hiv-viruses/: [ 148118889 ],
  https://zgcapan.wordpress.com/2017/12/20/call-for-papers-and-panels-section-21-historical-international-relations-12th-eisa-pan-european-conference-on-international-relations-september-2018/: [ 23245885 ]
}
https://www.theguardian.com/education

https://www.washingtonpost.com/entertainment/museums/obamas-portraits-unveiled-for-americans-presidents-exhibition/2018/02/12/d9f3691a-1000-11e8-8ea1-c1d91fcec3fe_story.html?utm_term=.634910d00200 --> [ 105255002, 14879942, 34162519 ]
http://thehill.com/homenews/administration/373441-the-federal-programs-trump-proposes-cutting-in-2019-budget?amp&__twitter_impression=true --> [ 105255002, 20550728 ]
https://nyti.ms/2jlWPWD --> [ 105614472, 354729119, 8984352 ]
http://www.bbc.com/capital/story/20171204-the-compelling-case-for-working-a-lot-less?ocid=ww.social.link.twitter --> [ 105614472, 24582955 ]
https://www.theglobeandmail.com/opinion/am-i-a-bad-feminist/article37591823/ --> [ 105614472, 413257620 ]
https://www.theglobeandmail.com/opinion/this-is-a-seismic-moment-for-women-and-we-cant-grow-weary/article37588733/ --> [ 105614472, 19000490 ]
https://www.nytimes.com/2018/01/21/us/womens-march-voices.html?nytapp=true --> [ 105614472, 19153715 ]
http://www.bbc.com/news/blogs-news-from-els

https://www.buzzfeed.com/alexkantrowitz/twitter-offered-rt-15-of-its-total-share-of-us-elections?utm_term=.sw7RMz9MA#.bv76q9Nqe --> [ 1075579124, 55176705 ]
https://www.thedailybeast.com/ceo-of-hq-the-hottest-app-going-if-you-run-this-profile-well-fire-our-host --> [ 1075579124, 15348009, 17831187 ]
http://bit.ly/2mOOR9P --> [ 1075579124, 11730942, ..., 69194007, 7004572 ]
https://www.seattletimes.com/business/amazon/this-city-hall-brought-to-you-by-amazon/ --> [ 1075579124, 148118889, ..., 42470250, 5562792 ]
https://www.nytimes.com/2017/11/17/upshot/income-inequality-united-states.html --> [ 1075579124, 44006633 ]
https://www.politico.com/story/2017/12/06/homeless-veterans-benefits-trump-207781?cid=apn --> [ 1075579124, 55176705 ]
https://www.reuters.com/article/us-usa-trump-wells-fargo-exclusive/exclusive-wells-fargo-sanctions-are-on-ice-under-trump-official-sources-idUSKBN1E12Y5 --> [ 1075579124, 870328182 ]
http://www.huffingtonpost.com.au/entry/daily-stormer-nazi-style-guide_us_5

https://www.theverge.com/platform/amp/2018/1/26/16932350/ice-immigration-customs-license-plate-recognition-contract-vigilant-solutions?__twitter_impression=true --> [ 11360602, 18602735, ..., 2426218693, 420284118 ]
https://www.chicagoreader.com/Bleader/archives/2018/02/07/ten-women-accuse-northwestern-journalism-professor-alec-klein-of-sexual-harassment-and-assault --> [ 11360602, 16595825, 23402525, 242308086 ]
http://www.chicagotribune.com/news/local/breaking/ct-met-no-more-parcc-20180207-story.html --> [ 11360602, 144143564 ]
https://www.vox.com/platform/amp/2018/2/8/16993172/trump-regulation-immigrants-benefits-public-charge?utm_campaign=vox&utm_content=entry&utm_medium=social&utm_source=twitter&__twitter_impression=true --> [ 11360602, 313683437, 3910951103 ]
http://trib.in/2Bs4Bpg --> [ 11360602, 16595825 ]
http://bit.ly/2CkTjzI --> [ 11360602, 5357 ]
https://www.vox.com/platform/amp/policy-and-politics/2018/2/14/17013570/senate-immigration-proposal-charts?__twitter_impression=t

https://www.theguardian.com/science/2017/nov/05/donald-trump-accused-blocking-satellite-climate-change-research --> [ 11730942, 140966030, ..., 291486075, 38471570 ]
https://nyti.ms/2hQcFoX --> [ 11730942, 378822672 ]
http://www.washingtonexaminer.com/jeff-flake-no-doubt-id-support-a-democrat-over-roy-moore/article/2640568 --> [ 11730942, 24746074, 332346899 ]
http://www.latimes.com/opinion/op-ed/la-oe-rosenworcel-fcc-net-neutrality-repeal-20171122-story.html --> [ 11730942, 137733302, ..., 24633727, 3039019074 ]
https://arstechnica.com/tech-policy/2017/12/comcast-to-customers-just-trustus-about-changed-net-neutrality-pledges/ --> [ 11730942, 48082526 ]
http://talkingpointsmemo.com/muckraker/alabama-drivers-licenses-voter-id --> [ 11730942, 168170834 ]
http://nyti.ms/2AuLZ8E --> [ 11730942, 15098171, 381734721 ]
https://www.nytimes.com/2017/12/23/us/politics/trump-immigration.html?_r=0 --> [ 11730942, 153229972, ..., 72638046, 980865674 ]
https://www.propublica.org/article/trump-justic

...

http://bbc.in/2zz1UBV --> [ 27645239, 388134481 ]
https://amp.theguardian.com/environment/2017/nov/09/uk-will-back-total-ban-on-bee-harming-pesticides-michael-gove-reveals --> [ 27645239, 276688978 ]
http://nyti.ms/2E9WA7l --> [ 27645239, 41368680 ]
https://news.nationalgeographic.com/2018/01/animals-octopi-dolphins-death-oceans/ --> [ 27645239, 3000588429 ]
https://www.propublica.org/article/join-propublicas-new-project-to-work-with-local-newsrooms --> [ 27661276, 31162511 ]
https://nyti.ms/2xVhhAW --> [ 27661276, 6627232 ]
http://wapo.st/2zyTmHS?tid=ss_tw-bottom&utm_term=.58dbcffab90e --> [ 27661276, 284885853 ]
https://www.nytimes.com/2017/11/14/opinion/louis-ck-not-funny-harassment.html?em_pos=small&emc=edit_ty_20171115&nl=opinion-today&nl_art=1&nlid=77981217&ref=headline&te=1 --> [ 27661276, 29924665 ]
https://www.theatlantic.com/business/archive/2017/11/media-apocalypse/546935/ --> [ 27661276, 30174656, 305270930, 35720261 ]
https://nyti.ms/2oMHKQx --> [ 27661276, 3910951103 ]
ht

https://www.nytimes.com/2017/10/31/nyregion/police-shooting-lower-manhattan.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=a-lede-package-region&region=top-news&WT.nav=top-news --> [ 286439617, 583393798, 6170212, 813057413742034944 ]
http://www.cnn.com/2017/11/10/us/native-lives-matter/index.html --> [ 286439617, 44006633, 538285083 ]
https://www.curbed.com/2018/1/24/16925074/apartment-eviction-process-gentrification --> [ 286439617, 945971568 ]
https://www.vox.com/policy-and-politics/2018/2/13/17004108/trump-aides-legal-fees-firing-resignations --> [ 286439617, 39878985 ]
http://bit.ly/2AoZ928 --> [ 289651922, 55739265 ]
https://mobile.nytimes.com/2018/01/31/sports/soccer/geoff-cameron-stoke-city.html?referer=https://t.co/xPsXiKiPCw?amp=1#click%3Dhttps%3A%2F%2Ft.co%2FxPsXiKiPCw --> [ 289651922, 48762865 ]
https://www.dallasnews.com/opinion/commentary/2017/11/09/dear-sutherland-springs-deserve-apology-news-media --> [ 289738296, 335413129 ]
https://www.nytimes.c

https://fb.me/4coQpeQ1S --> [ 3246958814, 91170675 ]
https://nyti.ms/2hMZKV1 --> [ 32973, 359074742 ]
https://thetyee.ca/Opinion/2017/10/30/I-Left-Vancouver/ --> [ 32992891, 6271482 ]
https://www.nytimes.com/2017/12/28/us/politics/trump-interview-mueller-russia-china-north-korea.html --> [ 32992891, 332346899 ]
https://www.wsj.com/articles/tillerson-assures-trump-administration-wont-block-european-trade-with-iran-1508472241 --> [ 331113029, 3838807353 ]
https://medium.com/@Brian_Whit/automatons-and-autocrats-the-brave-new-world-of-saudi-arabia-acba7d8d2a10 --> [ 331113029, 517937578 ]
https://www.nytimes.com/2017/11/16/opinion/saudi-iran-strategy.html --> [ 331113029, 3838807353 ]
https://www.haaretz.com/israel-news/1.832387 --> [ 331113029, 714661629162602496 ]
https://nyti.ms/2ySHvVn --> [ 332346899, 73977188 ]
http://www.al.com/news/index.ssf/2017/11/gadsden_residents_say_moores_b.html#incart_river_mobileshort_home --> [ 332346899, 35720261, 705141626029502464, 73977188 ]
http://abc

http://latimes.com/bradylist --> [ 366211826, 6170212 ]
https://www.propublica.org/article/california-murder-suspect-atomwaffen-division-extremist-hate-group --> [ 366211826, 44462995 ]
http://wapo.st/2o11ddi?tid=ss_tw&utm_term=.7fc30937a571 --> [ 366211826, 907312806 ]
http://bit.ly/2DCDqJM --> [ 368112759, 37563842 ]
https://www.axios.com/cvs-considers-acquiring-aetna-2501985383.html --> [ 370219796, 463059217 ]
https://www.recode.net/2017/11/30/16712284/tech-talent-startups-proposition-fang-companies-employment-jobs-engineering-design --> [ 372487283, 5357 ]
http://www.nydailynews.com/new-york/new-bill-require-social-worker-police-precinct-article-1.3705674 --> [ 374724416, 6545702 ]
http://theweek.com/speedreads/733917/military-officials-wanted-armed-drone-supporting-niger-mission-but-washington-said-no --> [ 381734721, 895951062 ]
https://www.nytimes.com/2017/12/29/world/canada/google-toronto-city-future.html --> [ 38228284, 603385690 ]
http://bit.ly/2huoFMh --> [ 3838807353, 5179

https://www.nytimes.com/2018/01/14/business/detroit-auto-show.html --> [ 55739265, 618556806 ]
http://foreignpolicy.com/2017/10/26/state-department-scraps-sanctions-office/ --> [ 564663702, 73977188 ]
https://www.buzzfeed.com/ilyazhegulev/russia-kaspersky-antivirus --> [ 564663702, 91170675 ]
http://www.nature.com/news/give-researchers-a-lifetime-word-limit-1.22835 --> [ 572479189, 91126624 ]
https://mobile.nytimes.com/2017/12/11/nyregion/explosion-times-square.html?action=click&module=TopStories&pgtype=Homepage --> [ 583393798, 895951062 ]
https://www.yahoo.com/music/cyntonia-brown-life-sentence-sparks-141514557.html --> [ 61272585, 9444512 ]
https://nyti.ms/2hs7oa4 --> [ 6170212, 68832423 ]
http://bit.ly/2yYEgxW --> [ 618556806, 62363146 ]
https://www.fastcompany.com/40490142/if-cars-are-weapons-then-safe-streets-are-the-best-counter-terrorism?utm_content=buffer53be3&utm_medium=social&utm_source=facebook.com&utm_campaign=buffer --> [ 618556806, 62363146 ]
https://www.brookings.edu/bl

In [32]:
log("total: " + str(total), logger)
log("countOfGreaterThan1: " + str(countOfGreaterThan1), logger)
log("Mean users amount per news: " + str(total / len(newsUsersMapping)), logger)

total: 23878
countOfGreaterThan1: 1878
mean: 1.1613248382860757


In [33]:
# We insert the newsUsersMapping:
for url, ids in pb(newsUsersMapping.items(), logger=logger, message="Inserting user list in all news rows"):
    newsCollection.updateOne({"url": url}, {"$set": {"users": ids}})
tt.tic("newsUsersMapping inserted")

Inserting user list in all news rows   0% [                    ]
Inserting user list in all news rows   9% [=                   ] (57.423s left)
Inserting user list in all news rows  19% [===                 ] (50.643s left)
Inserting user list in all news rows  29% [=====               ] (1m 6.387s left)
Inserting user list in all news rows  39% [=======             ] (52.174s left)
Inserting user list in all news rows  49% [=========           ] (1m 6.316s left)
Inserting user list in all news rows  59% [===========         ] (47.965s left)
Inserting user list in all news rows  69% [=============       ] (37.103s left)
Inserting user list in all news rows  79% [===============     ] (23.075s left)
Inserting user list in all news rows  89% [=================   ] (11.117s left)
Inserting user list in all news rows  99% [=================== ] (0.005s left)
Inserting user list in all news rows 100% [====================] (total duration: 1m 45.64s, mean duration: 0.005s)
--> tic: 18m 52.

1132.48

In [37]:
# Searching news that has no users:
toDeleteNews = []
for _id, row in newsCollection.items():
    if not dictContains(row, 'users') or len(row['users']) == 0:
        toDeleteNews.append(row['url'])
bp(toDeleteNews, logger)
log(str(len(toDeleteNews) / len(newsCollection) * 100) + " % of news to delete because no user shared it.", logger)

[
  http://wwd.com/business-news/media/conde-nast-to-close-teen-vogue-cut-80-jobs-and-lower-mag-frequenc,
  https://www.yahoo.com/amphtml/music/cyntonia-brown-life-sentence-sparks-141514557.html,
  ...,
  https://fb.me/1s6zYkAzE,
  https://tinyurl.com/ybxb5zm4
]
852.3809523809524 % of news to delete because no user shared it.


In [40]:
# Deleting news:
for url in toDeleteNews:
    newsCollection.delete({'url': url})

In [43]:
len(usersCollection)

500